In [1]:
from braceexpand import braceexpand
import numpy as np

In [15]:
paths = "/home/siddhesh1793/data/mmc4_converted/embeddings/docs_no_face_shard_{0..4}_v2.npy"
paths = list(braceexpand(paths))

In [18]:
embeddings_shape = [0, 768]

for path in paths:
    embeddings = np.load(path, allow_pickle=True)
    embeddings_shape[0] += len(embeddings)
embeddings_shape

[127192, 768]

In [17]:
# Indices to load data from
indices = np.random.randint(0, embeddings_shape[1], 500)

## Create one memmaped file and read

In [20]:
def create_memmap(paths, embeddings_shape, filename="/tmp/embeddings_map"):
	embeddings_map = np.memmap(filename, dtype=np.float32, mode="w+", shape=tuple(embeddings_shape))
	idx = 0
	for path in paths:
		embeddings = np.load(path, allow_pickle=True)
		embeddings_map[idx:idx+len(embeddings)] = embeddings
		idx += len(embeddings)
	del embeddings_map

In [21]:
def read_embeddings_from_memmap(filename, indices, embeddings_shape):
	embeddings_map = np.memmap(filename, dtype=np.float32, mode="r", shape=tuple(embeddings_shape))

	for idx in indices:
		yield embeddings_map[idx]

In [22]:
%timeit read_embeddings_from_memmap("/tmp/embeddings_map", indices, embeddings_shape)

75 ns ± 0.609 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [26]:
def read_each_shard_using_load(paths, indices, load_func, **kwargs):
	for index in indices:
		shard_idx = indices // 500
		index = indices % 500
		embeddings = load_func(paths[shard_idx], **kwargs)
		yield embeddings[index]

In [28]:
%timeit read_each_shard_using_load(paths, indices, np.load, allow_pickle=True)

134 ns ± 0.524 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [30]:
%timeit read_each_shard_using_load(paths, indices, np.lib.format.open_memmap, mode='r')

153 ns ± 0.608 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [31]:
%timeit read_each_shard_using_load(paths, indices, np.memmap, mode='r')